In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

print(torch.cuda.is_available())
!nvidia-smi

False
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
from torchvision import datasets
import torchvision.transforms as transforms

folder = "data"
trainset = datasets.CIFAR10(folder, train=True, download=True)
testset = datasets.CIFAR10(folder, train=False, download=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [3]:
trainset.data.shape

(50000, 32, 32, 3)

In [5]:
mu = torch.Tensor([0.4914, 0.4822, 0.4465])
sigma = torch.Tensor([0.2470, 0.2435, 0.2616])

In [6]:
trainset = datasets.CIFAR10(
    folder,
    train=True,
    download=False,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize(mu, sigma)]),
)

testset = datasets.CIFAR10(
    folder,
    train=False,
    download=False,
    transform=transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize(mu, sigma),
        ]
    ),
)

In [16]:
class MLP(nn.Module):
    def __init__(self, ninputs, nhidden, nclasses):
        super().__init__()
        self.fc1 = nn.Linear(ninputs, nhidden)
        self.fci = nn.Linear(nhidden, nhidden)
        self.fc2 = nn.Linear(nhidden, nclasses)

    def forward(self, x):
        out = torch.tanh(self.fc1(x))
        out = torch.tanh(self.fci(out))
        out = self.fc2(out)
        return out  # logits

device = 'cpu'
torch.manual_seed(0)
img, label = trainset[0]
img = img.view(-1)
ninputs = len(img)
nhidden = 512
nclasses = 10
model = MLP(ninputs, nhidden, nclasses).to(device)

In [8]:
def compute_accuracy(model, loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            imgs = imgs.view(imgs.shape[0], -1)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
    return correct / total


In [9]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

In [18]:
acc_train = compute_accuracy(model, train_loader)
acc_val = compute_accuracy(model, val_loader)
print("Accuracy of the randomly initialized model over the train and test sets: ", [acc_train, acc_val])

Accuracy of the randomly initialized model over the train and test sets:  [0.1054, 0.1072]


In [17]:
img.shape

torch.Size([3072])

In [19]:
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
n_epochs = 20

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        imgs = imgs.view(imgs.shape[0], -1)
        outputs = model(imgs)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

      # At end of each epoch
    acc_val = compute_accuracy(model, val_loader)
    loss_train_batch = float(loss)
    print(f"Epoch {epoch}, Batch Loss {loss_train_batch}, Val acc {acc_val}")

Epoch 0, Batch Loss 2.2864432334899902, Val acc 0.3894
Epoch 1, Batch Loss 2.199500322341919, Val acc 0.4082
Epoch 2, Batch Loss 2.1354150772094727, Val acc 0.4204
Epoch 3, Batch Loss 2.071441173553467, Val acc 0.4304
Epoch 4, Batch Loss 1.9980281591415405, Val acc 0.4367


KeyboardInterrupt: ignored